In [ ]:
%pip install semantic-link-labs --quiet

In [ ]:
import sempy.fabric as fabric
import sempy
import sempy_labs as labs
from sempy_labs import admin
from sempy_labs import migration, directlake
from sempy_labs import lakehouse as lake
from sempy_labs import report as rep
from sempy_labs.tom import connect_semantic_model


import pandas as pd
import requests
from azure.identity import ClientSecretCredential
import json
import os

In [ ]:
# Enter the service principal details
tenant_id = "Enter your Tenant Id"
client_id = "Enter your Client Id"
client_secret = "Enter your Secret Id" # make sure to store your secret e.g. in Azure Key Vault

# Get the object Id from Microsoft Entra Security Group using Microsoft Graph
#### The Service Principal needs some API permission to get this object id. Check the details here: https://www.data-traveling.com/articles/part-2-microsoft-fabric-admin-adding-security-groups-to-workspaces-with-semantic-link-and-python

In [ ]:
# This function creates the token access 
def get_access_token(tenant_id, client_id, client_secret):
    url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    body = {
        'client_id': client_id,
        'scope': 'https://graph.microsoft.com/.default',
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    response = requests.post(url, headers=headers, data=body)
    response_data = response.json()

    # Check if the response contains 'access_token'
    if 'access_token' in response_data:
        return response_data['access_token']
    else:
        # Print the error details for debugging
        error_description = response_data.get('error_description', 'No error description provided.')
        error = response_data.get('error', 'No error code provided.')
        print(f"Error obtaining access token: {error} - {error_description}")
        return None

# Function to get all groups
def get_all_groups(access_token):
    url = "https://graph.microsoft.com/v1.0/groups"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    groups = []
    while url:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            groups.extend(data.get('value', []))
            url = data.get('@odata.nextLink')  # URL for the next page of groups, if any
        else:
            print(f"Failed to fetch groups: {response.status_code} - {response.text}")
            break
    return groups


# This function gets the group id based on the name
def get_group_id(group_name, access_token):
    url = f'https://graph.microsoft.com/v1.0/groups?$filter=displayName eq \'{group_name}\''
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    groups = response.json().get('value', [])
    if groups:
        return groups[0]['id']  # Assuming the first match is the desired group
    return None

In [ ]:
# User input: enter the Microsoft Entra Id group
ad_pbi = 'Enter the security group name'

In [ ]:
# Call the function to get the access token
access_token = get_access_token(tenant_id, client_id, client_secret)

In [ ]:
# Get the object id from the respective security group
entraid_identifier = get_group_id(ad_pbi, access_token)

## Add the security group using semantic-link-labs

In [ ]:
# Add the securtiy group as a member of this workspace
labs.add_user_to_workspace(
    email_address = entraid_identifier,
    role_name = "Member",
    principal_type = "Group"
)

In [ ]:
# Print the users / security groups that have access to this workspace
labs.list_workspace_users()